<a href="https://colab.research.google.com/github/simonaron1111/GA_algoritmusok/blob/main/Simon_%C3%81ron_GA_bev_algoritmusok_colab_2_alap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# BIOINFORMATIKA - deceptív fitneszfüggvény gyakorlati alkalmazása
**Feladat:** deceptív fitneszfüggvény bevezetés és gyakorlati alkalmazás

---

Deceptív függvény:
* olyan fitness függvény, amelyben a lokálisan növekvő sémák (schema-average fitness) a GA-t a globális optimumtól távolodó irányba vezetik, így a GA jellemzően rossz lokális optimumhoz konvergál.


* globális maximum: 0000
* bármely alacsonyabb rendű szkémára teljesül: ha az összes 0-t 1-re cseréljük, akkor a szkéma átlagfitnesze nő.

* Készítünk egy 8 bites függvényt:
* f8​(x1​,…,x8​)=f4​(x1​,…,x4​)+f4​(x5​,…,x8​), ahol:
* az első 4 bit első blokknak, az utolsó 4 bit második blokknak felel meg.

Globális maximum:
* Az első blokk maximuma 0000, a másodiké is 0000,
* ezért az egész 8 bites maximumpont: 00000000.

Miért deceptív? (építőkocka-értelemben)
* Minden 4 bites blokk külön-külön deceptív (ezt az első projekt feladatban kell majd bizonyítani)
* Egy 8 bites sémánál, ha egy blokkban kicseréljük a 0-kat 1-re, akkor a blokk átlag-fitnesze nő → a teljes függvény blokkra eső része is nő.
* Így a GA-t bitenként, blokkonként „félrevezeti”: a lokális mintázat az 1-esek irányába tol, miközben a globális optimum a teljesen 0-s vektor.

---



## LABOR / FELADATOK CÉLJA

*   Genetikus algoritmus alapelveinek megértése
*   Szkémák (schema) és átlagfitnesz fogalmának gyakorlása
*   Deceptív fitness táj viselkedésének megismerése
*   Kódolás és problémareprezentáció szerepének megértése
*   Paraméterérzékenység vizsgálata
*   Lokális vs. globális optimum fogalmának elmélyítése
*   Kísérleti (empirikus) gondolkodás fejlesztése
*   Kódolási és implementációs készségek fejlesztése Pythonban
*   Kritikus gondolkodás az algoritmusválasztásról

## FELADAT: KÓD MEGÉRTÉS, majd gyakorlatok extra feladatokkal
A saját implementációhoz adok egy alap kódót, hogy ha vki elakadna, legyen minimál segítség, de ez legyen csak iránymutatás a saját megoldáshoz.

In [76]:
from itertools import product

# 4 bites Whitley-féle DF2 (globális max: 1111, lokális: 0000)
DF2 = {
    "0000": 28, "0001": 26, "0010": 24, "0011": 18,
    "0100": 22, "0101":  6, "0110": 14, "0111":  0,
    "1000": 20, "1001": 12, "1010": 10, "1011":  2,
    "1100":  8, "1101":  4, "1110":  6, "1111": 30,
}

def bit_not(bits: str) -> str:
    return "".join("1" if b == "0" else "0" for b in bits)

# 4 bites deceptív függvény, ahol a globális maximum 0000
f4 = {b: DF2[bit_not(b)] for b in DF2}

def f8(bits: str) -> int:
    """8 bites deceptív fitness: két 4 bites blokk összege."""
    assert len(bits) == 8
    return f4[bits[:4]] + f4[bits[4:]]

# Ellenőrzés: keressük meg a globális maximumot
BITSTRINGS_8 = ["".join(b) for b in product("01", repeat=8)]
values = {x: f8(x) for x in BITSTRINGS_8}
best_val = max(values.values())
best_points = [x for x, v in values.items() if v == best_val]

print("Globális maximum érték (8 bit):", best_val)
print("Globális maximum helye(i):", best_points)  # elvárt: ['00000000']


Globális maximum érték (8 bit): 60
Globális maximum helye(i): ['00000000']


## MELEGÍTŐ FELADAT (könnyű)



1.   Futtassátok a fenti kódot, győződjetek meg róla, hogy a globális maximum tényleg 00000000.
2.   Írjatok GA-t, és figyeljék meg, hogy tipikusan „1-esek felé” sodródik a populáció, nem pedig a 00000000 irányába.





In [103]:
import random
def random_individual(format = 8):
    return ''.join(random.choice('01') for _ in range(format))

def initialize_population(size, format = 8):
    return [random_individual(format) for _ in range(size)]

def tournament_selection(population, fitnesses, k=3):
    indices = random.sample(range(len(population)), k)
    best_index = max(indices, key=lambda i: fitnesses[i])
    return population[best_index]

def crossover(parent1, parent2, crossover_rate=0.7):
    if random.random() < crossover_rate:
        point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    return parent1, parent2

def mutate(individual, mutation_rate=0.01):
    chars = list(individual)
    for i in range(len(chars)):
        if random.random() < mutation_rate:
            chars[i] = '1' if chars[i] == '0' else '0'
    return ''.join(chars)

def run_genetic_algorithm(POP_SIZE, GENERATIONS, MUTATION_RATE, CROSSOVER_RATE, format=8, print_results = True, noise = False):

    population = initialize_population(POP_SIZE, format)

    if print_results:
      print(f"{'Gen':<5} | {'Max Fit':<10} | {'Best Individual':<15} | {'Avg Fit'}")
      print("-" * 50)

    for generation in range(GENERATIONS):
        if format == 8:
          fitnesses = [f8(ind) for ind in population]
        else:
          fitnesses = [f4[ind] for ind in population]
        if noise:
          fitnesses = [fitness + random.random() * 2.0 - 1.0 for fitness in fitnesses]
        max_fit = max(fitnesses)
        avg_fit = sum(fitnesses) / len(fitnesses)
        best_ind = population[fitnesses.index(max_fit)]

        if print_results:
          print(f"{generation:<5} | {max_fit:<10} | {best_ind:<15} | {avg_fit:.2f}")

        if max_fit == 8:
            if print_results:
              print(f"\nSolution found in generation {generation}!")
            break

        next_generation = []
        while len(next_generation) < POP_SIZE:
            p1 = tournament_selection(population, fitnesses, 3)
            p2 = tournament_selection(population, fitnesses, 3)

            c1, c2 = crossover(p1, p2, CROSSOVER_RATE)

            c1 = mutate(c1, MUTATION_RATE)
            c2 = mutate(c2, MUTATION_RATE)

            next_generation.extend([c1, c2])

        population = next_generation[:POP_SIZE]
        if format == 8:
          fitnesses = [f8(ind) for ind in population]
        else:
          fitnesses = [f4[ind] for ind in population]
        if noise:
          fitnesses = [fitness + random.random() * 2.0 - 1.0 for fitness in fitnesses]
    else:
        if print_results:
          print("\nMax generations reached.")
    return best_ind
POP_SIZE = 20
GENERATIONS = 50
MUTATION_RATE = 0.05
CROSSOVER_RATE = 0.8
run_genetic_algorithm(POP_SIZE, GENERATIONS, MUTATION_RATE, CROSSOVER_RATE)

Gen   | Max Fit    | Best Individual | Avg Fit
--------------------------------------------------
0     | 30         | 11101111        | 15.40
1     | 30         | 11101111        | 18.95
2     | 32         | 11111111        | 25.75
3     | 32         | 11111111        | 27.95
4     | 32         | 11111111        | 30.70
5     | 32         | 11111111        | 31.00
6     | 32         | 11111111        | 30.85
7     | 32         | 11111111        | 31.10
8     | 32         | 11111111        | 30.45
9     | 32         | 11111111        | 30.70
10    | 32         | 11111111        | 29.70
11    | 32         | 11111111        | 31.75
12    | 32         | 11111111        | 30.70
13    | 32         | 11111111        | 30.20
14    | 32         | 11111111        | 29.95
15    | 32         | 11111111        | 30.15
16    | 32         | 11111111        | 30.90
17    | 32         | 11111111        | 30.05
18    | 32         | 11111111        | 30.25
19    | 32         | 11111111        | 31.30
20

'11111111'

## 1) FELADAT (könnyű) – Táblázat kitöltése 4 biten

*   Írd fel táblázatba az összes 4 bites kromoszómát, a hozzájuk tartozó 𝑓4 fitneszértéket.
*   Rendezd őket csökkenő sorrendbe a fitness szerint.
*   Jelöld, hol van a globális maximum és hol a „csapda” (lokális optimum).



In [78]:
print("----- Original -----")
for block, fitness in f4.items():
  print(block, fitness)

sorted_f4 = dict(sorted(f4.items(), key=lambda x: x[1], reverse=True))
print("----- Sorted -----")
for block, fitness in sorted_f4.items():
  print(block, fitness)

----- Original -----
0000 30
0001 6
0010 4
0011 8
0100 2
0101 10
0110 12
0111 20
1000 0
1001 14
1010 6
1011 22
1100 18
1101 24
1110 26
1111 28
----- Sorted -----
0000 30
1111 28
1110 26
1101 24
1011 22
0111 20
1100 18
1001 14
0110 12
0101 10
0011 8
0001 6
1010 6
0010 4
0100 2
1000 0


In [79]:
# globalis maximum a 30, 0000-ban
# lokalis maximum pl a 20 a 0111-ben

## 2) FELADAT (könnyű) – Séma-átlag kézzel

*   Válaszd a sémát: 0***, számold ki kézzel a hozzá tartozó átlagfitneszt.
*   Utána nézd meg a 1*** sémát, és hasonlítsd össze.
*   Kövesd végig még 2–3 sémára (pl. 00** → 11**, * 0 * 0 → * 1 * 1 ).


In [80]:
DF2 = {
    "0000": 28, "0001": 26, "0010": 24, "0011": 18,
    "0100": 22, "0101":  6, "0110": 14, "0111":  0,
    "1000": 20, "1001": 12, "1010": 10, "1011":  2,
    "1100":  8, "1101":  4, "1110":  6, "1111": 30,
}
# 0***
# -> 0000, 0001, 0010, 0011, 0100, 0101, 0110, 0111
# -> 28 + 26 + 24 + 18 + 22 + 6 + 14 + 0
# = 138
# -> atlag 17.25 (/8)
# 1***
# -> 1000, 1001, 1010, 1011, 1100, 1101, 1110, 1111
# -> 20 + 12 + 10 + 2 + 8 + 4 + 6 + 30
# = 92
# -> atlag 11.5

# 00**
# -> 0000, 0001, 0010, 0011
# -> 28 + 26 + 24 + 18
# = 96
# -> atlag 24
# 11**
# -> 1100, 1101, 1110, 1111
# -> 8 + 4 + 6 + 30
# = 48
# -> atlag 12

# *0*0
# -> 0000, 0010, 1000, 1010
# -> 28 + 24 + 20 + 10
# = 82
# -> atlag 20.5
# *1*1
# -> 0101, 0111, 1101, 1111
# -> 6 + 0 + 4 + 30
# = 40
# -> atlag 10

## 3) FELADAT (könnyű) – GA egy generáció kézzel

*   Adj meg egy 6 elemű populációt 4 bites kromoszómákkal.
*   Számold ki a fitness értékeket, majd **roulette-wheel szelekcióval** válaszd ki 6 utód „szülőt”.
*   Nincs keresztezés, nincs mutáció – csak nézd meg, hogyan tolódik a populáció eloszlása.

Megjegyzés: Roulette-szelekcióra korábban adtam már példát. Azt lehet használni, mint inspirációt.



In [81]:
population = ['1000', '0110', '1010', '0111', '0010', '1101']
fitnesses = [f4[x] for x in population]
total_fitness = sum(fitnesses)
probabilities = [f / total_fitness for f in fitnesses]
next_generation = random.choices(population, weights=probabilities, k=6)
next_generation

['1010', '0110', '1101', '0111', '0111', '0010']

## 4) FELADAT (közepes) – GA implementáció 4 bitre


*   Írj egy egyszerű GA-t 4 bitre (pl. populáció=10, generációk=20).
*   Keresztezés: egypontos, mutáció: bit-flip 1%.
*   Figyeld meg, hányszor találja meg a 0000-t 50 futásból (statisztika).



In [82]:

POP_SIZE = 10
GENERATIONS = 20
MUTATION_RATE = 0.01
CROSSOVER_RATE = 0.8
success = 0
for run in range(0, 50):
  best_ind = run_genetic_algorithm(POP_SIZE, GENERATIONS, MUTATION_RATE, CROSSOVER_RATE, 4, False)
  if best_ind == '0000':
    success = success + 1
print(f"Found '0000' {success}/50 times")


Found '0000' 19/50 times


## 5) FELADAT (közepes) – Sémaszintű vizsgálat (8 bit)


*   A f8 függvényhez válaszd a sémát: ****0000.
*   Számold ki az átlagfitneszt.
*   Majd cseréld az összes 0-t 1-re: ****1111, és számold ki az átlagot.
*   Értelmezd az eredményt: mit „lát” a GA az utolsó 4 bitről?




In [83]:
def hatvany_halmaz(symbols, length):
  def _hatvany_halmaz(k, symbols, length, sol):
    if k == length:
      return [sol]
    result = []
    for symbol in symbols:
      result.extend(_hatvany_halmaz(k + 1, symbols, length, sol + symbol))
    return result
  return _hatvany_halmaz(0, symbols, length, "")

def flip_zeros_to_ones(schema):
  return ''.join(['1' if letter == '0' else letter for letter in schema])

def avg_fitness_of_schema(symbols, schema):
  fitting_strings = []
  for string in hatvany_halmaz(symbols, len(schema)):
    matches = True
    for i in range(len(schema)):
      if schema[i] != '*' and schema[i] != string[i]:
        matches = False
        break
    if matches:
      fitting_strings.append(string)
  if len(fitting_strings) == 0:
    return 0
  return sum([f8(string) for string in fitting_strings]) / len(fitting_strings)

In [84]:
sema_fitnessz = avg_fitness_of_schema(['0', '1'], '****0000')
flippelt_fitnessz = avg_fitness_of_schema(['0', '1'], flip_zeros_to_ones('****0000'))
print(f'elso: {sema_fitnessz}, masodik: {flippelt_fitnessz}')

elso: 44.375, masodik: 42.375


In [85]:
# A GA azt latja, hogy az utolso 4 bitnek nagyobb a fitnessze, ha azok 0-k

## 6) FELADAT (közepes) – Mutáció hatása


*   Ugyanazzal a GA-val (4 vagy 8 bit) futtass két szimulációt:
*   (a) 𝑝𝑚𝑢𝑡 = 0.001
*   (b) 𝑝𝑚𝑢𝑡 = 0.05
*   Hasonlítsd össze, melyik esetben könnyebb a 0000 illetve 00000000 megtalálása.
*   Írd le 3–4 mondatban, hogy miért.




In [90]:
POP_SIZE = 10
GENERATIONS = 20
MUTATION_RATE = 0.001
CROSSOVER_RATE = 0.8
success = 0
for run in range(0, 500):
  best_ind = run_genetic_algorithm(POP_SIZE, GENERATIONS, MUTATION_RATE, CROSSOVER_RATE, 8, False)
  if best_ind == '00000000':
    success = success + 1
print(f"{MUTATION_RATE=} Found '00000000' {success}/500 times")

POP_SIZE = 10
GENERATIONS = 20
MUTATION_RATE = 0.05
CROSSOVER_RATE = 0.8
success = 0
for run in range(0, 500):
  best_ind = run_genetic_algorithm(POP_SIZE, GENERATIONS, MUTATION_RATE, CROSSOVER_RATE, 8, False)
  if best_ind == '00000000':
    success = success + 1
print(f"{MUTATION_RATE=} Found '00000000' {success}/500 times")

MUTATION_RATE=0.001 Found '00000000' 24/500 times
MUTATION_RATE=0.05 Found '00000000' 40/500 times


In [87]:
# intuitiven, a nagyobb mutacios eselynek tobb eselye van megtalalni a helyes 00000000 megoldast, mivel a GA "szemeiben" az 1-esek tobbet ernek.

## 7) FELADAT (haladó) – Mutáció hatása


*   Készíts egy módosított fitness-függvényt:
*   f4′​(x)=f4​(x)+zaj, ahol a zaj pl. −1,0,+1−1,0,+1 egyenletes eloszlásból.
*   Vizsgáld meg, hogy a zaj hatására javul-e vagy romlik-e a GA esélye a globális optimum elérésére.

Javaslat: futtass 100 szimulációt zajjal és zaj nélkül.



In [99]:
POP_SIZE = 10
GENERATIONS = 20
MUTATION_RATE = 0.01
CROSSOVER_RATE = 0.8
success = 0
for run in range(0, 500):
  best_ind = run_genetic_algorithm(POP_SIZE, GENERATIONS, MUTATION_RATE, CROSSOVER_RATE, 8, False)
  if best_ind == '00000000':
    success = success + 1
print(f"NO NOISE Found '00000000' {success}/500 times")

success = 0
for run in range(0, 500):
  best_ind = run_genetic_algorithm(POP_SIZE, GENERATIONS, MUTATION_RATE, CROSSOVER_RATE, 8, False, True)
  if best_ind == '00000000':
    success = success + 1
print(f"WITH NOISE Found '00000000' {success}/500 times")

NO NOISE Found '00000000' 32/500 times
WITH NOISE Found '00000000' 34/500 times


In [ ]:
# a zaj segit a optimum megtalalasara

## 8) FELADAT (haladó) – Más kódolás vizsgálata


*   Definiálj egy **grayscale-szerű leképezést**: 0000 → 0, 1111 → 15, és egy **unimodális függvényt** ezen az intervallumon (pl. konkáv parabolát).
*   Mutasd meg, hogy ugyanarra az optimumra egy másik kódolás már nem deceptív, a GA könnyen találja a megoldást.
*   Hasonlítsd össze futási eredményekkel.



In [104]:

def f8_deceptive(bits):
    return f4[bits[:4]] + f4[bits[4:]]

def f8_unimodal(bits):
    val1 = int(bits[:4], 2)
    val2 = int(bits[4:], 2)

    score1 = 225 - (val1 - 15)**2
    score2 = 225 - (val2 - 15)**2

    return score1 + score2

def _random_individual():
    return ''.join(random.choice('01') for _ in range(8))

def _initialize_population(size):
    return [_random_individual() for _ in range(size)]

def _tournament_selection(population, fitness_func, k=3):
    candidates = random.sample(population, k)
    return max(candidates, key=fitness_func)

def _crossover(parent1, parent2, crossover_rate=0.7):
    if random.random() < crossover_rate:
        point = random.randint(1, 7)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    return parent1, parent2

def _mutate(individual, mutation_rate=0.01):
    chars = list(individual)
    for i in range(len(chars)):
        if random.random() < mutation_rate:
            chars[i] = '1' if chars[i] == '0' else '0'
    return ''.join(chars)

def run_ga(fitness_func, name):
    POP_SIZE = 20
    GENERATIONS = 50
    MUTATION_RATE = 0.05
    CROSSOVER_RATE = 0.8

    population = _initialize_population(POP_SIZE)

    print(f"\n--- {name} Futtatása ---")
    print(f"{'Gen':<5} | {'Max Fit':<10} | {'Legjobb Egyed':<15}")
    print("-" * 40)

    for generation in range(GENERATIONS):
        fitnesses = [fitness_func(ind) for ind in population]
        max_fit = max(fitnesses)
        best_ind = population[fitnesses.index(max_fit)]

        print(f"{generation:<5} | {max_fit:<10} | {best_ind:<15}")

        next_generation = []
        while len(next_generation) < POP_SIZE:
            p1 = _tournament_selection(population, fitness_func)
            p2 = _tournament_selection(population, fitness_func)

            c1, c2 = _crossover(p1, p2, CROSSOVER_RATE)

            c1 = _mutate(c1, MUTATION_RATE)
            c2 = _mutate(c2, MUTATION_RATE)

            next_generation.extend([c1, c2])

        population = next_generation[:POP_SIZE]

run_ga(f8_deceptive, "DECEPTÍV (Csapda)")
run_ga(f8_unimodal, "UNIMODÁLIS (Parabola)")


--- DECEPTÍV (Csapda) Futtatása ---
Gen   | Max Fit    | Legjobb Egyed  
----------------------------------------
0     | 54         | 00001101       
1     | 54         | 00001101       
2     | 58         | 00001111       
3     | 58         | 00001111       
4     | 58         | 00001111       
5     | 58         | 00001111       
6     | 58         | 00001111       
7     | 58         | 00001111       
8     | 58         | 00001111       
9     | 58         | 00001111       
10    | 58         | 00001111       
11    | 58         | 00001111       
12    | 58         | 00001111       
13    | 58         | 00001111       
14    | 58         | 00001111       
15    | 58         | 00001111       
16    | 58         | 00001111       
17    | 58         | 00001111       
18    | 58         | 00001111       
19    | 58         | 00001111       
20    | 58         | 00001111       
21    | 58         | 00001111       
22    | 58         | 00001111       
23    | 58         | 00001111     

## 9) FELADAT (haladó) – 8 bites blokkok kombinálása


*   A fenti 𝑓8-at módosítsd úgy, hogy az első blokkon f4, a második blokkon pedig egy egyszerű **hill-climbing-barát függvény** fusson (pl. az 1-ek száma). Mit is takar pontosan a **hill-climbing-barát függvény**? Kérem kifejteni.
*   Figyeld meg, hogy a GA az egyik blokkban „elcsúszik” (deceptív), a másikban jól dolgozik.
*  Írd le, mit tanulhatunk ebből a problémakódolásról és a GA korlátairól.



In [107]:
POP_SIZE = 10
GENERATIONS = 20
MUTATION_RATE = 0.001
CROSSOVER_RATE = 0.8
success = 0


def f8(bits: str) -> int:
    assert len(bits) == 8
    return f4[bits[:4]] + sum(int(bit) for bit in bits[4:])

best_ind = run_genetic_algorithm(POP_SIZE, GENERATIONS, MUTATION_RATE, CROSSOVER_RATE, 8)

Gen   | Max Fit    | Best Individual | Avg Fit
--------------------------------------------------
0     | 27         | 11011011        | 11.00
1     | 27         | 11011011        | 15.60
2     | 27         | 11011011        | 23.60
3     | 27         | 11011011        | 27.00
4     | 27         | 11011011        | 27.00
5     | 27         | 11011011        | 27.00
6     | 27         | 11011011        | 27.00
7     | 27         | 11011011        | 26.90
8     | 27         | 11011011        | 27.00
9     | 27         | 11011011        | 27.00
10    | 27         | 11011011        | 27.00
11    | 27         | 11011011        | 27.00
12    | 27         | 11011011        | 27.00
13    | 27         | 11011011        | 27.00
14    | 27         | 11011011        | 27.00
15    | 27         | 11011011        | 27.00
16    | 27         | 11011011        | 27.00
17    | 28         | 11011111        | 27.10
18    | 28         | 11011111        | 27.20
19    | 28         | 11011111        | 27.30

M

In [ ]:
# ez a kodolas eselyt ad ra, hogy a GA folyaman egy lokalis optimumba, tehat "csapdaba" essen a fejlodes
# a deceptiv fuggveny elleni megoldas arra kell, hogy mindig a globalis optimum fele menjen, ne alljon meg ahol epp jo